# Balancing and day ahead electricity prices. 
**Can day ahead predict balancing prices?** 

**Purpose:** In this exercise we will attempt to answer this question taking data from ENTSOE fitting and testing some time series models.

**Author:** Jordi Rof

**Date:** 20/03/2024

Note: You may also need to insall 'ipykernel' to make Jupyter work.

## 1. Import packages

In [ ]:
# Import packages
import numpy as np
import pandas as pd
#import statsmodels.api as sm
import statsmodels as sm
import matplotlib as plt
import re # for string substitution

## 2. Import and clean data
In this exercise we will work with data obtained from ENTSOE.
The data consist of 'day ahead' wholesale electricity prices and 'balancing' prices (aFRR).
We see that we have day ahead prices and balancing prices, as expected.
We have more than 35K observations that correspond to 2023 (every 15 minutes)
We also see tha the order is not correct (pay attention to the hours at the bottom and the top!)

In [ ]:
# Read data
data = pd.read_csv('dayahead_imbalance.csv')
# Explore dimensions
print(data.columns)
print(data.index)
print(data.Datetime)

In [ ]:
"""
In the printout shown above we can see that:
1. The text sting 'Datetime' does not have a standard format.
2. The order is not correct.
3. Some of the time features could be used as dummies that may contain important information of the behaviour of the vairables. Let's extract them.
"""
# Create time variables and order data correctly
data['Year'] = data.Datetime.str.slice(0,4).astype(int)
data['Month'] = data.Datetime.str.slice(5,7).astype(int)
data['Day'] = data.Datetime.str.slice(8,10).astype(int)
data['Hour'] = data.Datetime.str.slice(11,13).astype(int)
data['Minute'] = data.Datetime.str.slice(14,16).astype(int)
# Create a column with datetime format
data['Time'] = data.Datetime.str.slice(0,16)
data['Time'] = pd.to_datetime(data.Time, format='%Y-%m-%d %H:%M')
# Sort by time
data = data.sort_values('Time', ascending=True)
# Drop the original Datetime
data = data.drop('Datetime', axis = 1)
# We will also use the oportunity for giving the data frame modeling friendly names
data = data.rename(columns=lambda x: re.sub('_Day Ahead','_da',x))
data.columns = data.columns.str.lower()
# View newly created columns and a couple of extra variables
print(data[['time','year','month','day','hour','minute','austria_da', 'long_austria']].head(10))

In [ ]:
"""
In the printout shown above we can see that:
1. Obervation 0 is not present for balancing prices. We can simply remove it.
2. Day ahead prices have NAs. This is because this variable is hourly, while the balancing prices are every 15 minutes. We can solve this with gap filling (carrying over the last observed value).
3. The index is disorganized
"""
# Removing observation 0
data = data.drop(0, axis = 0)
# Filling missing values propagating the last valid value forward
data = data.fillna(method='ffill')
# Reset the index so it looks nicer
data = data.reset_index()
# Let's visualize some day ahead and balancing prices
da_sample_list = ['austria_da', 'netherlands_da', 'belgium_da']
bl_sample_list = ['long_austria', 'long_netherlands', 'long_belgium']
print(data[da_sample_list].head(10))
print(data[bl_sample_list].head(10))


## 3. Exploring and understanding the data

In [ ]:
# Start by describing some of the day ahead and balancing prices
print(data[da_sample_list].describe())
print(data[bl_sample_list].describe())


In [ ]:
"""
In the printout shown above we can see that:
1. Sufficient observations
2. Similar means in day ahead and balancing
3. Minimum and maximums are consistently higher in balancing than in day ahead
"""
# Let's continue with the correlation matrix (by country)
print(data[['austria_da', 'long_austria', 'short_austria']].corr())
print(data[['netherlands_da', 'long_netherlands', 'short_netherlands']].corr())
print(data[['belgium_da', 'short_belgium', 'short_belgium']].corr())


In [ ]:
"""
In the printout shown above we can see that:
1. We see that the correlation between day ahead and balancing is positive, but not strong. There may be a signal though.
2. In the case of Austria and Belgium short and long are the same data (symmetry?)
"""
# We can try to visualize it with scatterplots and time serie splot
data.plot.scatter(x = 'austria_da', y = 'long_austria', s = 100)
data.plot.scatter(x = 'netherlands_da', y = 'long_netherlands', s = 100)
data.plot.scatter(x = 'belgium_da', y = 'long_belgium', s = 100)
data[['long_austria','austria_da']].plot()
data[['long_netherlands','netherlands_da']].plot()
data[['short_netherlands','netherlands_da']].plot()
data[['long_belgium','belgium_da']].plot()

In [ ]:
"""
In the plots shown above we can see that:
1. The correlation between day ahead and balancing prices is visible.
2. The volatility of balancing prices is evident.
3. Means and standard deviations are quite stable over time.
4. Volatility of balancing prices in the netherlands seems to be higher in H2, consistently reaching prices above 1500 eur.
"""

## 4. Modelling dataset
In this section we add all the variables we need to the dataset.
- Level differences
- Lags
- Intercept
- Trends
- Dummies

In [ ]:
# Let's differentiate the data (not log differences as )
data_diff1 = data.diff(periods=1)
data_diff4 = data.diff(periods=4)
data_diff1 = data_diff1.add_suffix("_diff1",axis=1)
data_diff4 = data_diff4.add_suffix("_diff4",axis=1)
# Let's add the laggs of the differences
data_diff1_lag1 = data_diff1.shift(1)
data_diff4_lag1 = data_diff4.shift(1)
data_diff1_lag1 = data_diff1_lag1.add_suffix("_lag1",axis=1)
data_diff4_lag1 = data_diff4_lag1.add_suffix("_lag1",axis=1)
data_diff1_lag2 = data_diff1.shift(2)
data_diff4_lag2 = data_diff4.shift(2)
data_diff1_lag2 = data_diff1_lag2.add_suffix("_lag2",axis=1)
data_diff4_lag2 = data_diff4_lag2.add_suffix("_lag2",axis=1)
data_diff1_lag3 = data_diff1.shift(3)
data_diff4_lag3 = data_diff4.shift(3)
data_diff1_lag3 = data_diff1_lag3.add_suffix("_lag3",axis=1)
data_diff4_lag3 = data_diff4_lag3.add_suffix("_lag3",axis=1)
# Add dummies for some of the categorical variables
dummies = pd.get_dummies(data.hour, dtype=float)
# Concatenate the different dataframes created
data = pd.concat([data,data_diff1,data_diff4,
                  data_diff1_lag1,data_diff4_lag1,
                  data_diff1_lag2,data_diff4_lag2,
                  data_diff1_lag3,data_diff4_lag3,
                  dummies],axis=1)
# Add intercept and trens
data['intercept'] = 1 # Add intercept for the regression
data['lin_trend'] = data.index.values # Linear trend
data['log_trend'] = np.log(data.index.values^2) # Add intercept for the regression
# Drop all NAs so we do not have problems when fitting the models
data = data.dropna()
# Let's divide the series between the traning (or estimation) and test data. We will leave december out.
data_train = data[(data.month < 12)].copy()
data_test = data[(data.month == 12)].copy()
# Check out the training and test data
print(dummies.head(10))
print(data_train[["time","austria_da","long_austria","long_austria_diff1"]].head(5))
print(data_test[["time","austria_da","long_austria","long_austria_diff1"]].head(5))
print(data_diff1[["long_austria_diff1","long_belgium_diff1"]].head(10))
print(len(data.index)) # Check the amount of variables is still as expected
# Visualize the trend
data[['log_trend']].plot()

## 5. Modelling
In this section we start exploring some basic time series models.
We will focus first on a country and see how well we can predict balancing prices.

### 5.1. Austria ISF

In [ ]:
# Let's run a quick stationarity test of the Austrian variables
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html
# The null hypothesis of the Augmented Dickey-Fuller is that there is a unit root, with the alternative that there is no unit root. 
# If the pvalue is above a critical size, then we cannot reject that there is a unit root.
print(sm.tsa.stattools.adfuller(data_train.austria_da_diff1.dropna()))
print(sm.tsa.stattools.adfuller(data_train.long_austria_diff1.dropna()))

In [ ]:
"""
In the printout shown above we can see that:
1. We are goood. It seems that our data is stationary.
"""
# Let's try to fit a simple AR(1) model
model1a = sm.tsa.ar_model.AutoReg(data_train.long_austria_diff1.to_numpy(),1).fit()
out = 'AIC: {0:0.3f}, HQIC: {1:0.3f}, BIC: {2:0.3f}'
print('\n Model 1a')
print(out.format(model1a.aic, model1a.hqic, model1a.bic))
print(model1a.params)
print(model1a.pvalues)

# Let's try the same model running the standard OLS command
X =  data_train.long_austria_diff1
Y1 = data_train[['intercept','long_austria_diff1_lag1']]
model1b = sm.regression.linear_model.OLS(X, Y1).fit()
print('\n Model 1b')
print(model1b.summary())

In [ ]:
"""
In the printout shown above we can see that:
1. Model 1 and 2 are two different ways of fitting the same model.
2. In model 1/2. The past values of the balancing market seem to have some explanatory power, negative coefficient.
3. In model 1/2. The intercept is also strontly not significant.
4. In model 1/2. The R2 is rather low.
"""
# Let's explore trends in the data
Y2 = data_train[['intercept','long_austria_diff1_lag1','lin_trend','log_trend']]
model2 = sm.regression.linear_model.OLS(X, Y2).fit()
print('\n Model 2')
print(model2.summary())

In [ ]:
"""
In the printout shown above we can see that:
1. None of the trends that we have tried improves the result at all.
"""
# Let's explore other lags
Y3 = data_train[['intercept','long_austria_diff1_lag1','long_austria_diff1_lag2','long_austria_diff1_lag3']]
model3 = sm.regression.linear_model.OLS(X, Y3).fit()
print('\n Model 3')
print(model3.summary())

In [ ]:
""" 
In the printout shown above we can see that:
1. Adding more lags to the model clearly improves it, but we still do not get a good model.
    1.1. Better Adjusted R2 
    1.2. Better lower AIC
2. All the coefficients are still negative.
3. I would pick model 4 over any of the previous.
"""
# Now we try to add the day ahead prices (diff4) to see if they improve our fcast
Y4 = data_train[['intercept','long_austria_diff1_lag1','long_austria_diff1_lag2','long_austria_diff1_lag3','austria_da_diff4']]
model4 = sm.regression.linear_model.OLS(X, Y4).fit()
print('\n Model 4')
print(model4.summary())


In [ ]:
""" 
In the printout shown above we can see that:
1. Day ahead prices appear to be significant. Therefore, they carry information.
2. However, they do not improve any model selection metric (AIC, Adj.R2).
3. All in all 3 and 4 are the best, no big difference.
"""
# It is worth cehcking what happens with only DA prices
Y5 = data_train[['intercept','austria_da_diff4']]
model5 = sm.regression.linear_model.OLS(X, Y5).fit()
print('\n Model 5')
print(model5.summary())

In [ ]:
""" 
In the printout shown above we can see that:
1. DA only do not carry as much signal as the lags, even though they are significant.
2. All in all 3 and 4 are still the best.
"""
# Let's try another trick. Consider hours via dummies.
#data_train.columns.values
Y6 = data_train[['intercept','long_austria_diff1_lag1','long_austria_diff1_lag2','long_austria_diff1_lag3',7,8,12]]
Y6
model6 = sm.regression.linear_model.OLS(X, Y6).fit()
print('\n Model 6')
print(model6.summary())

In [ ]:
""" 
In the printout shown above we can see that:
1. Hourly dummies suggest that prices have some tendency to increase / decrease in some particular hours.
2. However, the effect is not strong, and it does not improve our model.
2. All in all, not a big difference between 3,4,6.
"""
# Let's take a step back and collect the ISF results of the models that we created
model_labels = ['model1','model2','model3','model4','model5','model6']
isf_results = pd.DataFrame()
#isf_results = isf_results.add_prefix('model_')
isf_results['metric'] = ['R2','adjR2','AIC/1000','DF model']
isf_results = isf_results.set_index('metric',drop=True)
loop_data = [model1b,model2,model3,model4,model5,model6]
isf_results['model1a'] = [np.nan,np.nan,model1a.aic/1000,np.nan]
for i in range(len(model_labels)):
    isf_results[model_labels[i]] = [loop_data[i].rsquared,
                                    loop_data[i].rsquared_adj,
                                    loop_data[i].aic/1000,
                                    loop_data[i].df_model]
# Print results
print(isf_results.round(2))

In [ ]:
"""
In the printout shown above we can see that:
1. Models 3, 4 and 6 are the best so far. Lags of balancing prices bring most of the predicitive power.
2. It is hard to add R2 with day ahead prices
3. It is hard to add R2 with hourly dummies
"""
# Tip for finding the parameters. Save the attributes into a list.
ls = []
for attr in dir(model1b):
    if not attr.startswith('_'):
        ls.append(attr)
print(ls)

### 5.1. Austria OOS
Let's try to predict december.

In [ ]:
# Let's analyze the out of sample predicion of the models
model1_oos_pred = model1b.predict(data_test[Y1.columns])
model2_oos_pred = model2.predict(data_test[Y2.columns])
model3_oos_pred = model3.predict(data_test[Y3.columns])
model4_oos_pred = model4.predict(data_test[Y4.columns])
model5_oos_pred = model5.predict(data_test[Y5.columns])
model6_oos_pred = model6.predict(data_test[Y6.columns])

# Let's first plot the results
oos = pd.concat([data_test.long_austria_diff1,
                 model1_oos_pred,model2_oos_pred,model3_oos_pred,
                 model4_oos_pred,model5_oos_pred,model6_oos_pred], axis=1)
oos.columns = ['long_austria_diff1'] + model_labels
for i in range(len(model_labels)):
    oos[['long_austria_diff1',model_labels[i]]].plot()

In [ ]:
"""
In the plots shown above we can see that:
1. The oos fit seems to be better in the models with high ISF.
"""
# Define some metrics for comfort
def rmse(x,y):
    op = ((x - y) ** 2).mean() ** .5
    return op
def mae(x,y):
    op = (abs(x - y)).mean()
    return op
def emean(x,y):
    op = (x - y).mean()
    return op
def emax(x,y):
    op = abs(x - y).max()
    return op
# Now let's take a look at some informative metrics
oos_results = pd.DataFrame()
oos_results['metric'] = ['r','RMSE','MAE','error mean', 'max. error']
oos_results = oos_results.set_index('metric',drop=True)
loop_data = [oos['model1'],oos['model2'],oos['model3'],oos['model4'],oos['model5'],oos['model6']]
for i in range(len(model_labels)):
    oos_results[model_labels[i]] =[oos['long_austria_diff1'].corr(loop_data[i]),
                            rmse(oos['long_austria_diff1'], loop_data[i]),
                            mae(oos['long_austria_diff1'], loop_data[i]),
                            emean(oos['long_austria_diff1'], loop_data[i]),
                            emax(oos['long_austria_diff1'], loop_data[i])]
# Print results
print(oos_results.round(2))

## Conclusions

### What have we learned?
1. We have only tried Austria, and this not necessarily applies to all the countires.
2. In Austria, day ahead prices appear to have some information about balancing prices, but they don't improve predictive power of the model a lot.
3. The same can be said about hourly dummies.
4. We are able to extrac much of the predictive power from lags.

### Does it make sense that day ahead prices have a low predictive power?
Yes. Balancing prices are a function of demand and supply mismatch in real time. In principle, there is no reason why day ahead prices should contain information about the mismatch. If the knowledge existed it would likely be used by traders.

### What else could be done?
1. With longer datasets, monthly effects could be explored.
2. Calendar effects may have some explanatory power.
3. Weather variables may have some explanatory power.
4. Electricity system data such as frequency may have some explanatory power.
5. Day ahead bid behaviour (no conensus amongst wind generators bid-offer?) could be a sensible gauge of uncertainty.